In [1]:
from abc import ABC, abstractmethod
from collections import namedtuple

In [2]:
Customer = namedtuple('Customer', 'name fidelity')
Customer

__main__.Customer

## Let's do it with less code, with functions as objects

In [3]:
class LineItem:
    
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
        
    def total(self):
        return self.price * self.quantity
    

class Order:
    
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = cart
        self.promotion = promotion
        
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)
        return self.total()-discount
        
    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

In [4]:
def fidelity_promo(order):
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

def bulk_order_promo(order):
    """10% discount for each lineitem with 20 or more units."""
    
    discount = 0
    for item in order.cart:
        if item.quantity >=20:
            discount += item.total() * .1

    return discount
    
def large_order_promo(order):
    """7% discount for orders with 10 or more discutinct items."""
       
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0

In [5]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)

In [6]:
cart = [LineItem('banana', 4, .5),
       LineItem('apple', 10, 1.5), 
       LineItem('watermellon', 5, 5.0)
       ]

In [8]:
Order(joe, cart, fidelity_promo)

<Order total: 42.00 due: 42.00>

## Best discount meta strategy

In [9]:
promos = [fidelity_promo, bulk_order_promo, large_order_promo]
def best_promo(order):
    return max(promo(order) for promo in promos)

In [10]:
Order(ann, cart, best_promo)

<Order total: 42.00 due: 39.90>

# Find all the promotions automatically

In [11]:
globals()

{'__name__': '__main__',
 '__doc__': 'Automatically created module for IPython interactive environment',
 '__package__': None,
 '__loader__': None,
 '__spec__': None,
 '__builtin__': <module 'builtins' (built-in)>,
 '__builtins__': <module 'builtins' (built-in)>,
 '_ih': ['',
  'from abc import ABC, abstractmethod\nfrom collections import namedtuple',
  "Customer = namedtuple('Customer', 'name fidelity')\nCustomer",
  "class LineItem:\n    \n    def __init__(self, product, quantity, price):\n        self.product = product\n        self.quantity = quantity\n        self.price = price\n        \n    def total(self):\n        return self.price * self.quantity\n    \n\nclass Order:\n    \n    def __init__(self, customer, cart, promotion=None):\n        self.customer = customer\n        self.cart = cart\n        self.promotion = promotion\n        \n    def total(self):\n        if not hasattr(self, '__total'):\n            self.__total = sum(item.total() for item in self.cart)\n        ret

In [13]:
help(globals)

Help on built-in function globals in module builtins:

globals()
    Return the dictionary containing the current scope's global variables.
    
    NOTE: Updates to this dictionary *will* affect name lookups in the current
    global scope and vice-versa.



In [15]:
promos = [globals()[name] for name in globals()
         if name.endswith('_promo')
         and name != 'best_promo']
promos

[<function __main__.fidelity_promo(order)>,
 <function __main__.bulk_order_promo(order)>,
 <function __main__.large_order_promo(order)>]

In [22]:
import inspect
help(inspect.getmembers)

Help on function getmembers in module inspect:

getmembers(object, predicate=None)
    Return all members of an object as (name, value) pairs sorted by name.
    Optionally, only return members that satisfy a given predicate.



In [26]:
inspect.getmembers(joe, predicate=inspect.ismethod)

[('__getnewargs__',
  <bound method Customer.__getnewargs__ of Customer(name='John Doe', fidelity=0)>),
 ('__repr__',
  <bound method Customer.__repr__ of Customer(name='John Doe', fidelity=0)>),
 ('_asdict',
  <bound method Customer._asdict of Customer(name='John Doe', fidelity=0)>),
 ('_make', <bound method Customer._make of <class '__main__.Customer'>>),
 ('_replace',
  <bound method Customer._replace of Customer(name='John Doe', fidelity=0)>)]